In [ ]:
!pip3 install -qU boto3 
!pip3 install -qU Pillow

In [ ]:
import json
import os
import glob
import sys
import boto3
import base64
import requests
import time
import pprint
from datetime import datetime
from IPython.display import Image, display

In [ ]:
#get modelARN
region = 'us-east-1'#'us-west-2' #
boto3_bedrock = boto3.client('bedrock',region)
boto3_bedrock.list_foundation_models()

In [ ]:
!wget https://cats.com/wp-content/uploads/2020/10/tabby-maine-coon-768x384.jpg

In [ ]:
stream = False
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [ ]:
#"modelId": "anthropic.claude-3-sonnet-20240229-v1:0"
# non streaming mode
def anthropic_claude_3(modelId,image_path,max_tokens,system_prompt):
    bedrock_runtime = boto3.client('bedrock-runtime')
    base64_image = encode_image(image_path)
    payload = {
        "modelId": modelId,
        "contentType": "application/json",
        "accept": "application/json",
        "body": {
            "anthropic_version": "bedrock-2023-05-31",
            "system": system_prompt,
            "max_tokens": max_tokens,
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "image",
                            "source": {
                                "type": "base64",
                                "media_type": "image/png",
                                "data": base64_image
                            }
                        },
                        {
                            "type": "text",
                            #"text": "Write me a detailed description of these two photos, and then a poem talking about it."
                            "text": f"这是我审查的结果："
                        }
                    ]
                }
            ]
        }
    }
	
    # Convert the payload to bytes
    body_bytes = json.dumps(payload['body']).encode('utf-8')
	
    # Invoke the model
    response = bedrock_runtime.invoke_model(
        body=body_bytes,
        contentType=payload['contentType'],
        accept=payload['accept'],
        modelId=payload['modelId']
    )
	
    # Process the response
    response_body = json.loads(response['body'].read().decode('utf-8'))
    pprint.pprint(response_body)
    #return round(end-start,2),response_body["usage"]["input_tokens"],response_body["usage"]["output_tokens"]
    return round(float(response['ResponseMetadata']['HTTPHeaders']['x-amzn-bedrock-invocation-latency'])/1000,2),int(response['ResponseMetadata']['HTTPHeaders']['x-amzn-bedrock-input-token-count']),int(response['ResponseMetadata']['HTTPHeaders']['x-amzn-bedrock-output-token-count'])
	

In [ ]:
#"modelId": "anthropic.claude-3-sonnet-20240229-v1:0"
# streaming mode
def anthropic_claude_3_stream(modelId,image_path,max_tokens,system_prompt):
    bedrock_runtime = boto3.client('bedrock-runtime')
    base64_image = encode_image(image_path)
    payload = {
        "modelId": modelId,
        "contentType": "application/json",
        "accept": "application/json",
        "body": {
            "anthropic_version": "bedrock-2023-05-31",
            "system": system_prompt,
            "max_tokens": max_tokens,
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "image",
                            "source": {
                                "type": "base64",
                                "media_type": "image/png",
                                "data": base64_image
                            }
                        },
                        {
                            "type": "text",
                            #"text": "Write me a detailed description of these two photos, and then a poem talking about it."
                            "text": f"这是我审查的结果："
                        }
                    ]
                }
            ],
            "temperature": 1,
            "top_p": 0.999,
            "top_k": 250,
"stop_sequences": ['\n\nHuman:']
        }
    }
	
    # Convert the payload to bytes
    body_bytes = json.dumps(payload['body']).encode('utf-8')
	
    # Invoke the model
    response = bedrock_runtime.invoke_model_with_response_stream(
        body=body_bytes, modelId=payload['modelId'], accept=payload['accept'], contentType=payload['contentType']
    )
    stream = response.get('body')
    chunk_obj = {}
	
    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                pprint.pprint(chunk_obj)
	
    # Process the response
    #response_body = json.loads(response['body'].read().decode('utf-8'))
    #pprint.pprint(response_body)
    {'type': 'message_stop', 'amazon-bedrock-invocationMetrics': {'inputTokenCount': 92, 'outputTokenCount': 277, 'invocationLatency': 3679, 'firstByteLatency': 677}}
	
    return round(float(chunk_obj['amazon-bedrock-invocationMetrics']['firstByteLatency'])/1000,2),round(float(chunk_obj['amazon-bedrock-invocationMetrics']['invocationLatency'])/1000,2),chunk_obj['amazon-bedrock-invocationMetrics']['inputTokenCount'],chunk_obj['amazon-bedrock-invocationMetrics']['outputTokenCount']

In [ ]:
# 调用结果
image_path = "tabby-maine-coon-768x384.jpg"
max_tokens = 200
system = """你是一个图片审核的AI专家，你需要协助人类进行图片审核的工作，你审核图片的标准如下
1. 图片需要是真人，女生，五官清晰，五官端正，图片包含有完整的脸部内容，
2. 图片不能够只露出眼睛，不能带面罩和口罩
2. 图片不能是动漫风格、不能男女混合（不能出现男性）、不能出现小孩、不能是纯风景
3. 图片不能太小，也就是图片的分辨率要高，同时尺寸不能小于480*768
4. 图片除非裸露了胸部和下体才可以判定为不通过，如果只是露出肩膀，露出肚脐，露出大腿和小腿这些都属于可以判定为通过，也就是整体构图不露骨就可以通过
5. 图片严禁涉及血腥、政治的题材，但是如果只是穿着和宗教风俗有关的衣服，其他方面没有问题，应该判为通过
你的输出请按照下面的json格式给出,如果不通过，请给出原因:
{ "结果" = "通过" or "不通过",
  "图片描述" = "xxxxxx",
  "不通过的原因" = "xxxx"
}
"""

In [ ]:
#haiku
print("Haiku:")
modelId = "anthropic.claude-3-haiku-20240307-v1:0"


In [ ]:
print(anthropic_claude_3(modelId,image_path,max_tokens,system))

In [ ]:
print(anthropic_claude_3_stream(modelId,image_path,max_tokens,system))	

In [ ]:
#sonnet
print("Sonnet:")
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"


In [ ]:
print(anthropic_claude_3(modelId,image_path,max_tokens,system))

In [ ]:
print(anthropic_claude_3_stream(modelId,image_path,max_tokens,system))		

In [ ]:
# 指定你的图片目录路径
directory_path = 'zhubo_moment_pics/disqualified'
# 创建图片文件的完整路径模式
pattern = os.path.join(directory_path, '*.jpg')

# 使用glob找到所有的.jpg文件
image_files = glob.glob(pattern)

# 如果没有找到文件，打印消息
if not image_files:
    print("No defined pattern files found in the directory.")
else:
    # 打印每一个文件的路径
    for image_file in image_files:
        display(Image(filename=image_file))
        print(anthropic_claude_3(modelId,image_file,max_tokens,system))
